In [1]:
%pip install -U torchtext==0.6

     ---------------------------------------- 0.0/64.2 kB ? eta -:--:--
     ---------------------------------------- 64.2/64.2 kB 1.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: C:\Users\farha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
%pip install spacy

     ---------------------------------------- 0.0/12.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.1 MB ? eta -:--:--
     --------------------------------------- 0.0/12.1 MB 435.7 kB/s eta 0:00:28
     --------------------------------------- 0.1/12.1 MB 544.7 kB/s eta 0:00:23
      --------------------------------------- 0.2/12.1 MB 1.5 MB/s eta 0:00:08
     -- ------------------------------------- 0.7/12.1 MB 3.5 MB/s eta 0:00:04
     ----- ---------------------------------- 1.8/12.1 MB 7.4 MB/s eta 0:00:02
     ----------- ---------------------------- 3.4/12.1 MB 11.9 MB/s eta 0:00:01
     ------------------- -------------------- 5.9/12.1 MB 18.1 MB/s eta 0:00:01
     ------------------------ --------------- 7.5/12.1 MB 20.9 MB/s eta 0:00:01
     ---------------------------- ----------- 8.7/12.1 MB 20.7 MB/s eta 0:00:01
     ------------------------------- -------- 9.5/12.1 MB 20.3 MB/s eta 0:00:01
     --------------------------------- ----- 10.4/12.1 MB 2


[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: C:\Users\farha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
import nltk
import re
from torchtext.data import Field, BucketIterator
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
import spacy
import random
from torch.utils.tensorboard import SummaryWriter

In [58]:
import string
import numpy as np
import re
from pickle import load,dump
from unicodedata import normalize
from numpy import array
from numpy.random import rand
from numpy.random import shuffle
from numpy import argmax
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
# from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu

In [3]:
def load_doc(filename):
    # open the file as read only
    file = open(filename, mode='rt', encoding='utf-8')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [4]:
def to_pairs(doc):
    lines = doc.strip().split('\n')
    pairs = [line.split('\t') for line in  lines]
    return pairs

In [5]:
def clean_pairs(lines):
    cleaned = list()
    # prepare regex for char filtering
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for pair in lines:
        clean_pair = list()
        for line in pair:
            # normalize unicode characters
            line = normalize('NFD', line).encode('ascii', 'ignore')
            line = line.decode('UTF-8')
            # tokenize on white space
            line = line.split()
            # convert to lowercase
            line = [word.lower() for word in line]
            # remove punctuation from each token
            line = [word.translate(table) for word in line]
            # remove non-printable chars form each token
            line = [re_print.sub('', w) for w in line]
            # remove tokens with numbers in them
            line = [word for word in line if word.isalpha()]
            # store as string
            clean_pair.append(' '.join(line))
        cleaned.append(clean_pair)
    return array(cleaned)

In [6]:
def save_clean_data(sentences, filename):
    dump(sentences, open(filename, 'wb'))
    print('Saved: %s' % filename)

In [7]:
# load dataset
filename = 'deu.txt'
doc = load_doc(filename)
# split into english-german pairs
pairs = to_pairs(doc)
# clean sentences
clean_pairs = clean_pairs(pairs)
# save clean pairs to file
save_clean_data(clean_pairs, 'english-german.pkl')

Saved: english-german.pkl


In [8]:
# load a clean dataset
def load_clean_sentences(filename):
    return load(open(filename, 'rb'))

In [9]:
raw_dataset = load_clean_sentences('english-german.pkl')
# reduce dataset size
n_sentences = 20000
dataset = raw_dataset[:n_sentences, :]
# random shuffle
shuffle(dataset)
# split into train/test
train, test = dataset[:14000], dataset[14001:n_sentences]
# save
save_clean_data(dataset, 'english-german-both.pkl')
save_clean_data(train, 'english-german-train.pkl')
save_clean_data(test, 'english-german-test.pkl')

Saved: english-german-both.pkl
Saved: english-german-train.pkl
Saved: english-german-test.pkl


In [10]:
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [11]:
def max_length(lines):
    return max(len(line.split()) for line in lines)

In [12]:
def encode_sequences(tokenizer, length, lines):
    X = tokenizer.texts_to_sequences(lines)
    # pad sequences with 0 values
    X = pad_sequences(X, maxlen=length, padding='post')
    return X
def encode_output(sequences, vocab_size):
    ylist = list()
    for sequence in sequences:
        encoded = to_categorical(sequence, num_classes=vocab_size)
        ylist.append(encoded)
    y = array(ylist)
    y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
    return y

In [13]:
def define_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
    model = Sequential()
    model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
    model.add(LSTM(n_units))
    model.add(RepeatVector(tar_timesteps))
    model.add(LSTM(n_units, return_sequences=True))
    model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
    return model

In [49]:
# load datasets
dataset = load_clean_sentences('english-german-both.pkl')
train = load_clean_sentences('english-german-train.pkl')
test = load_clean_sentences('english-german-test.pkl')
# prepare english tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
print('English Vocabulary Size: %d' % eng_vocab_size)
print('English Max Length: %d' % (eng_length))
# prepare german tokenizer
ger_tokenizer = create_tokenizer(dataset[:, 1])
ger_vocab_size = len(ger_tokenizer.word_index) + 1
ger_length = max_length(dataset[:, 1])
print('German Vocabulary Size: %d' % ger_vocab_size)
print('German Max Length: %d' % (ger_length))
# prepare training data
trainX = encode_sequences(ger_tokenizer, ger_length, train[:, 1]) # germana
trainY = encode_sequences(eng_tokenizer, eng_length, train[:, 0]) # engleza
trainY = encode_output(trainY, eng_vocab_size)
# prepare validation data
testX = encode_sequences(ger_tokenizer, ger_length, test[:, 1])
testY = encode_sequences(eng_tokenizer, eng_length, test[:, 0])
testY = encode_output(testY, eng_vocab_size)
# define model
model = define_model(ger_vocab_size, eng_vocab_size, ger_length, eng_length, 256)
model.load_weights('model.h5')

model.compile(optimizer='adam', loss='categorical_crossentropy')
# summarize defined model
print(model.summary())
# fit model
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model.fit(trainX, trainY, epochs=40, batch_size=256, validation_data=(testX, testY), callbacks=[checkpoint], verbose=2)

English Vocabulary Size: 3816
English Max Length: 6
German Vocabulary Size: 6192
German Max Length: 10
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 10, 256)           1585152   
                                                                 
 lstm_2 (LSTM)               (None, 256)               525312    
                                                                 
 repeat_vector_1 (RepeatVec  (None, 6, 256)            0         
 tor)                                                            
                                                                 
 lstm_3 (LSTM)               (None, 6, 256)            525312    
                                                                 
 time_distributed_1 (TimeDi  (None, 6, 3816)           980712    
 stributed)                                                      
                 

C:\Users\farha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 4.10138 to 3.76153, saving model to model.h5
55/55 - 14s - loss: 3.8944 - val_loss: 3.7615 - 14s/epoch - 246ms/step
Epoch 3/40

Epoch 3: val_loss improved from 3.76153 to 3.62429, saving model to model.h5
55/55 - 14s - loss: 3.6587 - val_loss: 3.6243 - 14s/epoch - 251ms/step
Epoch 4/40

Epoch 4: val_loss improved from 3.62429 to 3.54875, saving model to model.h5
55/55 - 15s - loss: 3.5406 - val_loss: 3.5488 - 15s/epoch - 269ms/step
Epoch 5/40

Epoch 5: val_loss improved from 3.54875 to 3.50196, saving model to model.h5
55/55 - 14s - loss: 3.4617 - val_loss: 3.5020 - 14s/epoch - 259ms/step
Epoch 6/40

Epoch 6: val_loss improved from 3.50196 to 3.47413, saving model to model.h5
55/55 - 15s - loss: 3.4066 - val_loss: 3.4741 - 15s/epoch - 269ms/step
Epoch 7/40

Epoch 7: val_loss improved from 3.47413 to 3.44958, saving model to model.h5
55/55 - 14s - loss: 3.3685 - val_loss: 3.4496 - 14s/epoch - 259ms/step
Epoch 8/40

Epoch 8: val_loss improved from 3.44958

In [50]:
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

In [51]:
def predict_sequence(model, tokenizer, source):
    prediction = model.predict(source, verbose=0)[0]
    integers = [argmax(vector) for vector in prediction]
    target = list()
    for i in integers:
        word = word_for_id(i, tokenizer)
        if word is None:
            break
        target.append(word)
    return ' '.join(target)

In [52]:
def evaluate_model(model, tokenizer, sources, raw_dataset):
    actual, predicted = list(), list()
    for i, source in enumerate(sources):
        # translate encoded source text
        source = source.reshape((1, source.shape[0]))
        translation = predict_sequence(model, eng_tokenizer, source)
        raw_target, raw_src = raw_dataset[i]
        if i < 10:
            print('src=[%s], target=[%s], predicted=[%s]' % (raw_src, raw_target, translation))
        actual.append([raw_target.split()])
        predicted.append(translation.split())
    # calculate BLEU score
    print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
    print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
    print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
    print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

In [53]:
print('train')
evaluate_model(model, eng_tokenizer, trainX, train)
# test on some test sequences
print('test')
evaluate_model(model, eng_tokenizer, testX, test)

train
src=[tom hat improvisiert], target=[tom improvised], predicted=[tom improvised]
src=[sie gab einen schuss ab], target=[she shot a gun], predicted=[she made a eyes]
src=[ich kann dich nicht zuruckhalten], target=[i cant stop you], predicted=[i cant see you]
src=[er liebt mich nicht], target=[he doesnt love me], predicted=[he likes to me]
src=[die zeit wirds zeigen], target=[time will tell], predicted=[please will do]
src=[ich hasse beerdigungen], target=[i hate funerals], predicted=[i hate french]
src=[er macht uberstunden], target=[he works long hours], predicted=[he was a]
src=[das ist richtig], target=[thats correct], predicted=[thats is]
src=[sie bringen mich noch um], target=[youre killing me], predicted=[youre me me]
src=[ich komme], target=[im coming], predicted=[im go]
BLEU-1: 0.543303
BLEU-2: 0.385699
BLEU-3: 0.293805
BLEU-4: 0.145651
test
src=[er lebt in osaka], target=[he lives in osaka], predicted=[he went at]
src=[ziehe eine nummer], target=[take a number], predicted=

In [26]:
from nltk.translate.bleu_score import corpus_bleu

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report


In [38]:
# Functia de traducere
def translate(text):
    text_to_translate = [text]

    sequences = ger_tokenizer.texts_to_sequences(text_to_translate)
    sequences = pad_sequences(sequences, maxlen=ger_length, padding='post')

    translation = predict_sequence(model, eng_tokenizer, sequences)
    return translation


In [46]:
source = "Ich bin h# eute schlechter Laune."
target = "I'm in a bad mood today."

output = translate(source)

# BLEU
bleu = corpus_bleu([target], [output], weights=(1.0, 0, 0, 0))
print(bleu)

0.3888888888888889


C:\Users\farha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\farha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\farha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-pa

In [48]:

source = "Es ist immer noch Montag."
target = "It's still Monday."

output = translate(source)
print(output)

# BLEU
bleu = corpus_bleu([target], [output], weights=(1.0, 0, 0, 0))
print(bleu)

its still quiet
0.3333333333333333


C:\Users\farha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\farha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\farha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-pa

In [127]:

# Avem modelul antrenat de Alice.
# Luam o parte din datasetul pe care s-a antrenat modelul si il testam cu acesta => label IN probe
# Luam o alta parte din dataset pe care modelul lui Alice NU s-a antrenat si il testam si pe acesta => label OUT probe
# Construim un clasificator care, pe baza scorurilor generate pe fiecare label, acesta poate eticheta daca o propozitie
# este IN probe sau OUT probe
# Pentru testarea clasificatorului, se aleg alte bucati din dataset care sunt atat IN probe si OUT probe si se verifica
# acuratetea ghicelii. Daca per total avem rezultat de 50% sau mai putin => ESEC , altfel, atacul este un success


raw_dataset = load_clean_sentences('english-german.pkl')

dimension_to_split =  14000
max_dim = 100000
# reduce dataset size
n_sentences_limit = 5000
dataset = raw_dataset[:dimension_to_split, :]
# random shuffle
shuffle(dataset)

dataset_trained = dataset[:n_sentences_limit, :]

# make dataset_trained an array where first 3 columns are arrays and 4th column is a number
#dataset_trained = np.array(dataset_trained)
#dataset_trained = np.reshape(dataset_trained, (len(dataset_trained), 4))


column_pahar = []
new_dataset_trained = []
for sentence in dataset_trained:
    sentence = np.append(sentence, translate(sentence[1]))
    sentence = np.append(sentence, 1) # IN
    column_pahar = sentence[0]
    sentence[0] = sentence[1]
    sentence[1] = column_pahar
    bleu = corpus_bleu([sentence[1]], [sentence[2]], weights=(1.0, 0, 0, 0))
    sentence = np.append(sentence, bleu)
    new_dataset_trained.append(sentence)

dataset_2 = raw_dataset[dimension_to_split:max_dim, :]
# random shuffle
shuffle(dataset_2)
dataset_not_trained = dataset_2[:n_sentences_limit, :]

new_dataset_untrained = []
for sentence in dataset_not_trained:
    sentence = np.append(sentence, translate(sentence[1]))
    sentence = np.append(sentence, 0) # OUT
    column_pahar = sentence[0]
    sentence[0] = sentence[1]
    sentence[1] = column_pahar
    bleu = corpus_bleu([sentence[1]], [sentence[2]], weights=(1.0, 0, 0, 0))
    sentence = np.append(sentence, bleu)
    new_dataset_untrained.append(sentence)

# big_list = [dataset_trained, dataset_not_trained]
big_list = new_dataset_trained + new_dataset_untrained
shuffle(big_list)
# df = pd.DataFrame(dataset_trained, dataset_not_trained, )


# We luam 5000 de propozitii din A_train (folosite de Alice la training)
# Luam 5000 propozitii nefolosite in training si alcatuim un tabel de forma
# PROP_GERMANA  OUTPUT  TARGET  LABEL

# PROP_GERMANA = propozitia in germana
# OUTPUT = propozitia in engleza generata de Alice
# TARGET = propozitia in engleza corecta
# LABEL = IN sau OUT



C:\Users\farha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\farha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\farha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-pa

In [128]:
import pandas as pd 
# print (new_dataset_trained)
df = pd.DataFrame(big_list, columns=['INPUT', 'REF', 'OUTPUT', 'LABEL', 'BLEU'])

In [129]:
filename = 'dataset.csv'
df.to_csv(filename, sep=',', index=False, encoding='utf-8')

In [130]:
print(df)

                                                  INPUT  \
0                          ich habe ein bisschen hunger   
1                                        lass nicht los   
2                        kannst du diesen stein anheben   
3                                     ich bin hundemude   
4                       ich bin nicht so schwer wie tom   
5                       sie wird gewiss uberrascht sein   
6                          das mache ich auf jeden fall   
7                                        er ubte verrat   
8                                       isst du fleisch   
9                               sie schuttelte den kopf   
10                       ich war krank vor sorge um tom   
11                              er scheint mude zu sein   
12                                         bleibt ruhig   
13                                       wie langweilig   
14                     es lohnt sich bestimmt zu warten   
15                denkst du dass tom mir verzeihen wird 

In [133]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report


In [134]:
# df - dataset, split between training and testing
# NAIVE BAYERS

df_train = df[:8000]
df_test = df[8000:]

X_train = df_train['BLEU']
y_train = df_train['LABEL']


# Train the classifier
classifier = MultinomialNB()
classifier.fit(X_train.values.reshape(-1, 1), y_train)

# Predict the labels for the testing data
y_pred = classifier.predict(df_test['BLEU'].values.reshape(-1, 1))

# Calculate the accuracy of the classifier
accuracy = accuracy_score(df_test['LABEL'], y_pred)
print("Accuracy:", accuracy)

# Generate classification report
report = classification_report(df_test['LABEL'], y_pred)
print("Classification Report:")
print(report)


Accuracy: 0.4985
Classification Report:
              precision    recall  f1-score   support

           0       0.50      1.00      0.67       997
           1       0.00      0.00      0.00      1003

    accuracy                           0.50      2000
   macro avg       0.25      0.50      0.33      2000
weighted avg       0.25      0.50      0.33      2000



C:\Users\farha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\farha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\farha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344

In [135]:
# RANDOM FOREST

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

X_train = df_train['BLEU']
y_train = df_train['LABEL']

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train.values.reshape(-1, 1), y_train)

# Predict the labels for the testing data
y_pred = clf.predict(df_test['BLEU'].values.reshape(-1, 1))

# Calculate the accuracy of the classifier
accuracy = accuracy_score(df_test['LABEL'], y_pred)
print("Accuracy:", accuracy)

# Generate classification report
report = classification_report(df_test['LABEL'], y_pred)

print("Classification Report:")
print(report)


Accuracy: 0.5475
Classification Report:
              precision    recall  f1-score   support

           0       0.53      0.73      0.62       997
           1       0.58      0.37      0.45      1003

    accuracy                           0.55      2000
   macro avg       0.56      0.55      0.53      2000
weighted avg       0.56      0.55      0.53      2000



In [136]:
# NEAREST NEIGHBOUR
from sklearn.neighbors import KNeighborsClassifier

X_train = df_train['BLEU']
y_train = df_train['LABEL']

neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train.values.reshape(-1, 1), y_train)

# Predict the labels for the testing data

y_pred = neigh.predict(df_test['BLEU'].values.reshape(-1, 1))

# Calculate the accuracy of the classifier
accuracy = accuracy_score(df_test['LABEL'], y_pred)
print("Accuracy:", accuracy)

# Generate classification report
report = classification_report(df_test['LABEL'], y_pred)

print("Classification Report:")
print(report)

Accuracy: 0.5175
Classification Report:
              precision    recall  f1-score   support

           0       0.52      0.41      0.46       997
           1       0.52      0.62      0.56      1003

    accuracy                           0.52      2000
   macro avg       0.52      0.52      0.51      2000
weighted avg       0.52      0.52      0.51      2000



In [137]:
# Perceptron

from sklearn.linear_model import Perceptron

X_train = df_train['BLEU']
y_train = df_train['LABEL']

clf = Perceptron(tol=1e-3, random_state=0)
clf.fit(X_train.values.reshape(-1, 1), y_train)

# Predict the labels for the testing data
y_pred = clf.predict(df_test['BLEU'].values.reshape(-1, 1))

# Calculate the accuracy of the classifier
accuracy = accuracy_score(df_test['LABEL'], y_pred)

print("Accuracy:", accuracy)

# Generate classification report
report = classification_report(df_test['LABEL'], y_pred)

print("Classification Report:")
print(report)


Accuracy: 0.501
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       997
           1       0.50      1.00      0.67      1003

    accuracy                           0.50      2000
   macro avg       0.25      0.50      0.33      2000
weighted avg       0.25      0.50      0.33      2000



In [138]:
# Multi-layer Perceptron (MLP)

from sklearn.neural_network import MLPClassifier

X_train = df_train['BLEU']
y_train = df_train['LABEL']

clf = MLPClassifier(random_state=1, max_iter=300).fit(X_train.values.reshape(-1, 1), y_train)

# Predict the labels for the testing data
y_pred = clf.predict(df_test['BLEU'].values.reshape(-1, 1))

# Calculate the accuracy of the classifier
accuracy = accuracy_score(df_test['LABEL'], y_pred)

print("Accuracy:", accuracy)

# Generate classification report
report = classification_report(df_test['LABEL'], y_pred)

print("Classification Report:")
print(report)




Accuracy: 0.541
Classification Report:
              precision    recall  f1-score   support

           0       0.53      0.63      0.58       997
           1       0.55      0.46      0.50      1003

    accuracy                           0.54      2000
   macro avg       0.54      0.54      0.54      2000
weighted avg       0.54      0.54      0.54      2000

